# **ACS EXTRACTION**

***What socioeconomic, limnological, ichthyological factors predict the success of stocked walleye populations in the midwest?***

---

In [ ]:
pip install census us

In [ ]:
from census import Census
from us import states
import pandas as pd

CENSUS_API_KEY = 'd0ea7d38656f63662436cf60d2a99a904d33775a'
c = Census(CENSUS_API_KEY)

# Target states
state_fips_map = {
    'WI': states.WI.fips,
    'MI': states.MI.fips,
    'OH': states.OH.fips,
    'IA': states.IA.fips,
    'MN': states.MN.fips,
    'SD': states.SD.fips,
    'NE': states.NE.fips
}

# Variable codes and their friendly names
variables = {
    'NAME': 'County_Name',
    'B19013_001E': 'Median_Income',
    'B17001_002E': 'Below_Poverty',
    'B01003_001E': 'Population',
    'B15003_022E': 'Edu_HS_Grad',
    'B15003_025E': 'Edu_Bachelors'
}

years = range(2009, 2023)
all_data = []

for year in years:
    print(f"\n📅 Year: {year}")
    for state_abbr, state_fips in state_fips_map.items():
        try:
            data = c.acs5.state_county(list(variables.keys()), state_fips, Census.ALL, year=year)
            if not data:
                print(f"⚠️  No data returned for {state_abbr}, {year}")
                continue

            for row in data:
                row['state_abbr'] = state_abbr
                row['year'] = year
                row['FIPS'] = row.get('state', '') + row.get('county', '')

            all_data.extend(data)
            print(f"✅ Pulled {len(data)} rows for {state_abbr}, {year}")

        except Exception as e:
            print(f"❌ Error in {state_abbr}, {year}: {e}")

# Convert to DataFrame
if not all_data:
    raise ValueError("No data was retrieved from the Census API. Check API key or parameters.")

df = pd.DataFrame(all_data)

# Rename columns
for old, new in variables.items():
    if old in df.columns:
        df.rename(columns={old: new}, inplace=True)

# Confirm final columns
expected_cols = ['FIPS', 'state_abbr', 'year'] + list(variables.values())
present_cols = [col for col in expected_cols if col in df.columns]

if not all(col in df.columns for col in ['FIPS', 'state_abbr', 'year']):
    print(f"\n🚨 Missing critical columns! Available columns:\n{df.columns.tolist()}")
    raise KeyError("Critical columns 'FIPS', 'state_abbr', or 'year' not found in the final DataFrame.")

# Final DataFrame
df = df[present_cols]
df.to_csv("acs_data_cleaned.csv", index=False)
print(f"\n✅ Success! Data saved. Shape: {df.shape}")
print(df.head())
